In [1]:
import numpy as np
from matplotlib import pyplot as plt
from hypergraph import hypergraph, read

In [2]:
data = 'contact-high-school'
path = data + '/' + data + '-'

In [3]:
t_min = 1386330122

C = read.read_data(data, t_min = t_min)
C = hypergraph.hypergraph(C)
G = hypergraph.projected_graph(C, as_hyper = True)

In [4]:
def adjacency_matrix(G):
    A = np.zeros((G.n, G.n))
    for f in G.C:
        A[f[0], f[1]] += 1
        A[f[1], f[0]] += 1
    return(A)

In [5]:
G.n, G.m

(268, 10026)

In [36]:
def experiment(G, n_stub, n_vertex, n_rounds, sample_after, message_every = 100):
    
    w = np.zeros((G.n, G.n))
    x = np.zeros((G.n, G.n))
    w2 = np.zeros((G.n, G.n))
    
    G.MH(label = 'stub', n_steps = n_stub, verbose = False, message = False)
    print('Completed stub-labeled MCMC with ' + str(int(n_stub)) + ' steps.')
    
    for i in range(n_rounds + sample_after):
        G.MH(label = 'vertex', n_steps = n_vertex, verbose = False, message = False)
        if i % message_every == 0:
            print("Round " + str(i) + " of " + str(n_rounds + sample_after) + ' completed, ' + str(int(i * n_vertex)) + ' steps taken, acceptance rate = ' + str(round(G.acceptance_rate, 4)))
        samples = i - sample_after
        if samples > 0:
            W = adjacency_matrix(G)
            X = 1.0*(W > 0)
            
            w = (samples - 1)/samples*w + 1.0/samples*W
            x = (samples - 1)/samples*x + 1.0/samples*X
            w2 = (samples -1)/samples*w2 + 1.0/samples*(W**2)
    return(w, x, w2)

In [ ]:
n_steps = round(G.m*np.log(G.m))
w, x, w2 = experiment(G, n_stub = 0 , n_vertex = n_steps/10, n_rounds = 100000, sample_after = 50, message_every = 1000)

Completed stub-labeled MCMC with 0 steps.
Round 0 of 100050 completed, 0 steps taken, acceptance rate = 0.2783


In [102]:
np.savetxt('throughput/contact-high-school/w.txt', w, fmt = "%.4f")
np.savetxt('throughput/contact-high-school/x.txt', x, "%.4f")
np.savetxt('throughput/contact-high-school/w2.txt', w2, "%.4f")